In [1]:
!pip install pyspark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("IPL Data Analysis") \
    .master("local[*]") \
    .getOrCreate()


In [3]:
from google.colab import files
uploaded = files.upload()


Saving ball_by_ball_data.csv to ball_by_ball_data.csv


In [5]:
df = spark.read.csv("ball_by_ball_data.csv", header=True, inferSchema=True)
df.show(5)
df.printSchema()


+---------+--------+-----------+-------+-----------+------------+------------+-----------+-----------+-----------+------+----------+--------------+----------------+----------+-----------------+---------+------------+----------+----------+------+----------+--------------+------------+------------+--------+------------+-----------+-------------+-------+
|season_id|match_id|     batter| bowler|non_striker|team_batting|team_bowling|over_number|ball_number|batter_runs|extras|total_runs|  batsman_type|     bowler_type|player_out|fielders_involved|is_wicket|is_wide_ball|is_no_ball|is_leg_bye|is_bye|is_penalty|wide_ball_runs|no_ball_runs|leg_bye_runs|bye_runs|penalty_runs|wicket_kind|is_super_over|innings|
+---------+--------+-----------+-------+-----------+------------+------------+-----------+-----------+-----------+------+----------+--------------+----------------+----------+-----------------+---------+------------+----------+----------+------+----------+--------------+------------+--------

In [6]:
print("Total Rows:", df.count())
print("Total Columns:", len(df.columns))
df.select("match_id").distinct().count()


Total Rows: 278205
Total Columns: 30


1169

In [7]:
from pyspark.sql.functions import sum

df.groupBy().sum("total_runs").show()


+---------------+
|sum(total_runs)|
+---------------+
|         374283|
+---------------+



In [10]:
from pyspark.sql.functions import col

df.groupBy("batter").sum("batter_runs") \
  .withColumnRenamed("sum(batter_runs)", "total_runs") \
  .orderBy(col("total_runs").desc()) \
  .show(10)


+--------------+----------+
|        batter|total_runs|
+--------------+----------+
|       V Kohli|      8671|
|     RG Sharma|      7048|
|      S Dhawan|      6769|
|     DA Warner|      6567|
|      SK Raina|      5536|
|      MS Dhoni|      5439|
|      KL Rahul|      5235|
|AB de Villiers|      5181|
|     AM Rahane|      5032|
|      CH Gayle|      4997|
+--------------+----------+
only showing top 10 rows



In [13]:
df.filter(df.player_out.isNotNull()) \
  .groupBy("bowler").count() \
  .orderBy(col("count").desc()) \
  .show(10)


+---------------+-----+
|         bowler|count|
+---------------+-----+
|       R Ashwin| 4868|
|      SP Narine| 4421|
|        B Kumar| 4378|
|      RA Jadeja| 4127|
|      YS Chahal| 3905|
|      PP Chawla| 3895|
|Harbhajan Singh| 3496|
|      JJ Bumrah| 3474|
|       A Mishra| 3444|
|       AR Patel| 3371|
+---------------+-----+
only showing top 10 rows



In [15]:
df.groupBy("batter").sum("batter_runs") \
  .withColumnRenamed("sum(batter_runs)", "total_runs") \
  .write.csv("batsman_runs_output", header=True)
